In [1]:
!mongoimport --db restaurants --collection restaurants --drop --file ./data/restaurants.json
!mongoimport --db restaurants --collection neighborhoods --drop --file ./data/neighborhoods.json

2021-10-07T17:57:37.042-0500	connected to: mongodb://localhost/
2021-10-07T17:57:37.047-0500	dropping: restaurants.restaurants
2021-10-07T17:57:38.124-0500	25359 document(s) imported successfully. 0 document(s) failed to import.
2021-10-07T17:57:38.282-0500	connected to: mongodb://localhost/
2021-10-07T17:57:38.282-0500	dropping: restaurants.neighborhoods
2021-10-07T17:57:39.346-0500	195 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
import json
import pymongo

In [3]:
HOST = "localhost"
PORT = "27017"
DB_NAME = "restaurants"

### Utilities functions

In [4]:
def transform_json(element):
    geo_element = dict()
    geo_element["type"] = "Feature"
    geo_element["geometry"] = dict()
    
    try:
        geo_element["geometry"]["type"] = element["location"]["type"]
        geo_element["geometry"]["coordinates"] = element["location"]["coordinates"]
    except KeyError:
        geo_element["geometry"]["type"] = element["geometry"]["type"]
        geo_element["geometry"]["coordinates"] = element["geometry"]["coordinates"]
    
    geo_element["properties"] = dict()
    geo_element["properties"]["name"] = element["name"]
    return geo_element

In [5]:
def save_json(data, filename = "ouput.json"):
    with open("./data/" + filename, "w") as outfile:
        json.dump(data, outfile, indent = 4, sort_keys = True)

In [6]:
def create_feature_collection(features):  
    feature_collection = dict()
    feature_collection["type"] = "FeatureCollection"
    feature_collection["features"] = features
    return feature_collection

### Connecting MongoDB and setting up for geographic queries

In [7]:
# Connecting to database
db = None
try:
    db_client = pymongo.MongoClient(HOST + ":" + PORT)
    db = db_client[DB_NAME]
    print("Conecction established successfully!")
except pymongo.errors.ConnectionFailure:
    print("Connection cannot be established")

Conecction established successfully!


In [8]:
# Creating geospatial indexes: https://docs.mongodb.com/manual/core/2dsphere/
db.restaurants.create_index([("location", "2dsphere")])
db.neighborhoods.create_index([("geometry", "2dsphere")])

'geometry_2dsphere'

### Making some basic queries

In [9]:
# Getting the first restaurant in restaurants collection
point = db.restaurants.find_one()
point

{'_id': ObjectId('55cba2476c522cafdb053ae1'),
 'location': {'coordinates': [-73.8803827, 40.7643124], 'type': 'Point'},
 'name': 'Brunos On The Boulevard'}

In [10]:
# Transforming the previous result to a GeoJSON string
geo_point = transform_json(point)
geo_point

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-73.8803827, 40.7643124]},
 'properties': {'name': 'Brunos On The Boulevard'}}

In [11]:
# Storing GeoJSON string to disk
save_json(geo_point, "geo_point.json")

In [12]:
# Getting the first neighborhood in neighborhoods collection
polygon = db.neighborhoods.find_one()

In [13]:
# Transforming the previous result to a GeoJSON string
geo_polygon = transform_json(polygon)
geo_polygon

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-73.89138023380261, 40.86170058826304],
    [-73.89106280613036, 40.86152941211661],
    [-73.89106059904508, 40.86147181520911],
    [-73.89115375546018, 40.86091361616668],
    [-73.89174395026582, 40.86017714781636],
    [-73.8922570762111, 40.85940997630237],
    [-73.89299354227633, 40.85838612046538],
    [-73.8944135507376, 40.85643377580147],
    [-73.89477596003171, 40.85592952563393],
    [-73.89549889768499, 40.854749765891384],
    [-73.89560883346181, 40.85457037009787],
    [-73.89644052566297, 40.85490775915559],
    [-73.89709926839446, 40.855172290942875],
    [-73.89831118626606, 40.85406115105141],
    [-73.89863543744137, 40.8539069754361],
    [-73.89816577520747, 40.85514637639308],
    [-73.89877598610725, 40.85536324709642],
    [-73.89967155800208, 40.8556689083972],
    [-73.90060386049306, 40.855987237865136],
    [-73.90077922747537, 40.856047109729],
    [-73.90085477134976, 40.8559374

In [14]:
# Storing GeoJSON string to disk
save_json(geo_polygon, "geo_polygon.json")

### Making geographical queries

In [15]:
# Neighborhood intersects point
intersection = db.neighborhoods.find_one({"geometry": {"$geoIntersects": {"$geometry": {"type": "Point", "coordinates": [-73.93414657, 40.82302903]}}}})

In [16]:
save_json(transform_json(intersection), "geo_intersection.json")

In [17]:
# Restaurants within neighborhood
neighborhood = db.neighborhoods.find_one({"geometry": {"$geoIntersects": {"$geometry": {"type": "Point", "coordinates": [-73.93414657, 40.82302903]}}}})
restaurants = db.restaurants.find({"location": {"$geoWithin": {"$geometry": neighborhood["geometry"]}}})

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append(transform_json(restaurant))
geo_feature_collection = create_feature_collection(geo_restaurants)

In [18]:
save_json(geo_feature_collection, "geo_feature_collection.json")

In [19]:
# Restaurants by proximity
restaurants = db.restaurants.find({"location": {"$geoWithin": {"$centerSphere": [[-73.93414657, 40.82302903 ], 1.5/3963.2]}}}) # 1.5 miles

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append(transform_json( restaurant))
save_json( create_feature_collection(geo_restaurants), "geo_proximity.json")

In [20]:
# Restaurants within sphere
restaurants = db.restaurants.find({"location": {"$nearSphere": {"$geometry": {"type": "Point", "coordinates": [-73.93414657, 40.82302903]}, "$minDistance": 1000, "$maxDistance": 1500}}})

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append(transform_json(restaurant))
save_json(create_feature_collection(geo_restaurants), "geo_sphere.json")